In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
from utils import load_json_config

In [3]:
config = load_json_config("config.json")

In [4]:
df = pd.read_csv(config["organized_dataset_name"])

In [5]:
!gdown {config["album_cover_imgs"]}

Downloading...
From (original): https://drive.google.com/uc?id=1LrygB3h79OvHPqOjkMHicoIthQ8DBfcy
From (redirected): https://drive.google.com/uc?id=1LrygB3h79OvHPqOjkMHicoIthQ8DBfcy&confirm=t&uuid=0d7a4f55-8ec0-4e71-9e8a-27ade4a78fd3
To: /home/yehor/Desktop/Genre_based_on_album_cover/album_covers.zip
100%|████████████████████████████████████████| 180M/180M [00:15<00:00, 11.6MB/s]


In [6]:
!unzip /home/yehor/Desktop/Genre_based_on_album_cover/album_covers.zip

Archive:  /home/yehor/Desktop/Genre_based_on_album_cover/album_covers.zip
   creating: album_covers/
  inflating: album_covers/63.jpg     
  inflating: album_covers/823.jpg    
  inflating: album_covers/2666.jpg   
  inflating: album_covers/2100.jpg   
  inflating: album_covers/1409.jpg   
  inflating: album_covers/189.jpg    
  inflating: album_covers/2114.jpg   
  inflating: album_covers/2672.jpg   
  inflating: album_covers/77.jpg     
  inflating: album_covers/837.jpg    
  inflating: album_covers/2882.jpg   
  inflating: album_covers/1353.jpg   
  inflating: album_covers/1435.jpg   
  inflating: album_covers/2128.jpg   
  inflating: album_covers/1421.jpg   
  inflating: album_covers/1347.jpg   
  inflating: album_covers/2896.jpg   
  inflating: album_covers/638.jpg    
  inflating: album_covers/2869.jpg   
  inflating: album_covers/1390.jpg   
  inflating: album_covers/604.jpg    
  inflating: album_covers/2699.jpg   
  inflating: album_covers/2841.jpg   
  inflating: album_covers

In [10]:
images_location = "/home/yehor/Desktop/Genre_based_on_album_cover/album_covers"

In [7]:
# image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for pretrained CNNs
    transforms.ToTensor(),         # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize for pretrained models
])

In [8]:
# Load and preprocess images
def preprocess_image(image_index):
    image = Image.open(f"{images_location}/{image_index}.jpg").convert('RGB')  # Ensure image is RGB
    return transform(image)

In [11]:
# Apply preprocessing
df['image_tensor'] = df['album_id'].apply(preprocess_image)

In [6]:
df

,album_id,genre,image_tensor
0,0,0,"[[[tensor(0.9646), tensor(0.9474), tensor(0.96..."
1,1,0,"[[[tensor(-1.2274), tensor(-1.1589), tensor(-1..."
2,2,1,"[[[tensor(2.2489), tensor(2.2489), tensor(2.24..."
3,4,0,"[[[tensor(-1.3644), tensor(-1.4500), tensor(-1..."
4,6,4,"[[[tensor(0.5022), tensor(0.5193), tensor(0.50..."
...,...,...,...
1603,2155,1,"[[[tensor(-2.1008), tensor(-2.1008), tensor(-2..."
1604,2157,2,"[[[tensor(-0.1143), tensor(-0.1143), tensor(-0..."
1605,2159,3,"[[[tensor(-0.2856), tensor(-0.2856), tensor(-0..."
1606,2160,4,"[[[tensor(2.1290), tensor(2.1633), tensor(0.89..."


In [12]:
df.to_csv(config["preprocessed_dataset_name"], index=False)

## DATASET:

In [14]:
class AlbumDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Get image and label
        image_path = (f"{images_location}/{self.dataframe.iloc[idx]['album_id']}.jpg")
        label = self.dataframe.iloc[idx]['genre']
        
        # Load and transform image
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        
        return image, label